# Aprendizaje por refuerzo

## Resolución de CartPole mediante Q-Learning

En este *notebook* se resolverá el entorno CartPole utilizando el algoritmo Q-Learning.

### Discretización

Una característica importante de CartPole es que sus estados son continuos, lo que, en principio, impide poder acceder y modificar la tabla Q (discreta) según estos valores.

La solución en este caso es cuantizar o discretizar los valores continuos. Para esto podemos dividir los posibles valores continuos en *rangos* (llamados *bins*).

Por ejemplo, si tenemos un estado continuo en el rango [0,1] y queremos discretizarlo en 10 posibles *bins*, estos quedarían como:

* Valores en [0, 0.1) = Bin 0
* Valores en [0.1, 0.2) = Bin 1
...
* Valores en [0.9,1] = Bin 9

De este modo, si el estado observado fuese **0.15** indexaríamos la tabla Q como si fuese **1**.

Para CartPole solo es necesario discretizar los estados pero esta misma técnica podría utilizarse también en el caso de que fuese necesario discretizar las acciones.



---



Como en el ejercicio anterior, hacemos uso de una serie de librerías para poder mostrar los resultados como vídeos:

In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1

!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

import gym
from gym import logger as gymlogger
from gym.wrappers.record_video import RecordVideo
gymlogger.set_level(40) #error only

import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else:
    print("No se encuentra el video")

def wrap_env(env):
  env = RecordVideo(env, './video',  episode_trigger = lambda episode_number: True)
  return env

Definimos ahora los parámetros de nuestro experimento. Nótese que el número de episodios es elevado: aprender mediante *aprendizaje por refuerzo* suele llevar muchísimos intentos.


In [ ]:
# Número de episodios
n_episodes = 2000

# Parametros de Q-Learning
alpha = 0.1
gamma = 0.99

# Parametros de epsilon-voraz
epsilon = 1.0
epsilon_min = 0.01
delta = 0.999

# Posibles bins por cada dimension del estado
bins_per_state = 16

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Ahora iteramos durante `n_episodes` episodios sobre el entorno, aprendiendo de cada experiencia según el algoritmo Q-Learning.

Iremos monitorizando las recompensas acumuladas de cada episodio, así como el promedio de recompensas acumuladas de los últimos 100 episodios (para tener una mejor estimación de la tendencia).

In [ ]:
import gym
import numpy as np
from collections import deque

bins = np.array([
    # posición del carro
    np.linspace(-2.4, 2.4, bins_per_state),
    # velocidad del carro
    np.linspace(-3.0, 3.0, bins_per_state),
    # ángulo del poste
    np.linspace(-0.5, 0.5, bins_per_state),
    # velocidad del poste
    np.linspace(-2.0, 2.0, bins_per_state)
])

# Inicializamos el entorno con el wrapper para hacer videos
env = wrap_env(gym.make('CartPole-v1'))

# Definimos el numero de posibles acciones para crear la tabla
n_actions = env.action_space.n # (2)

# La tabla tiene 4 dimensiones para el estado y 1 para las acciones
q_table = np.zeros((bins_per_state, bins_per_state, bins_per_state, bins_per_state, n_actions))

# Variable para monitorizar el promedio de ultimos 100 episodios
last_rewards = deque(maxlen=100)

# Escribimos una función que discretiza el estado (ver np.digitize)
def discretize(cont_state):
    state = []
    for i in range(len(cont_state)):
        state.append(np.digitize(cont_state[i], bins[i])-1)
    return tuple(state)

# Bucle para el número de episodios seleccionado
for i in range(n_episodes):
    episode_reward = 0

    state = env.reset()
    state = discretize(state)

    done = False
    while not done:
        env.render()

        # Politica eps-voraz
        if np.random.uniform() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        # Ejecutamos accion
        next_state, reward, done, _ = env.step(action)
        next_state = discretize(next_state)

        # Acumulamos recompensa
        episode_reward += reward

        # Actualizacion q_learning
        q_table[state][action] += alpha * (reward + gamma * np.max(q_table[next_state]) - q_table[state][action])

        # Preparamos para siguiente iteracion
        state = next_state

    # Agregamos la recompensa acumulada al histórico
    last_rewards.append(episode_reward)

    # Reportamos el rendimiento en el ultimo episodio
    print("ep.: {}, eps: {:.2f}, ret.: {}, ret. prom.: {:.1f}".format(i, epsilon, episode_reward, np.mean(last_rewards)))

    # Decaemos el epsilon para balancear la exploración-explotación
    if epsilon > epsilon_min:
        epsilon *= delta


ep.: 0, eps: 1.00, ret.: 9.0, ret. prom.: 9.0
ep.: 1, eps: 1.00, ret.: 15.0, ret. prom.: 12.0
ep.: 2, eps: 1.00, ret.: 46.0, ret. prom.: 23.3
ep.: 3, eps: 1.00, ret.: 47.0, ret. prom.: 29.2
ep.: 4, eps: 1.00, ret.: 12.0, ret. prom.: 25.8
ep.: 5, eps: 1.00, ret.: 29.0, ret. prom.: 26.3
ep.: 6, eps: 0.99, ret.: 15.0, ret. prom.: 24.7
ep.: 7, eps: 0.99, ret.: 15.0, ret. prom.: 23.5
ep.: 8, eps: 0.99, ret.: 12.0, ret. prom.: 22.2
ep.: 9, eps: 0.99, ret.: 13.0, ret. prom.: 21.3
ep.: 10, eps: 0.99, ret.: 24.0, ret. prom.: 21.5
ep.: 11, eps: 0.99, ret.: 22.0, ret. prom.: 21.6
ep.: 12, eps: 0.99, ret.: 42.0, ret. prom.: 23.2
ep.: 13, eps: 0.99, ret.: 16.0, ret. prom.: 22.6
ep.: 14, eps: 0.99, ret.: 23.0, ret. prom.: 22.7
ep.: 15, eps: 0.99, ret.: 24.0, ret. prom.: 22.8
ep.: 16, eps: 0.98, ret.: 21.0, ret. prom.: 22.6
ep.: 17, eps: 0.98, ret.: 23.0, ret. prom.: 22.7
ep.: 18, eps: 0.98, ret.: 29.0, ret. prom.: 23.0
ep.: 19, eps: 0.98, ret.: 30.0, ret. prom.: 23.4
ep.: 20, eps: 0.98, ret.: 10.0, 

KeyboardInterrupt: ignored

In [ ]:
# Visualiza el último episodio (desde el env.reset() hasta el env.close())
env.close()
show_video()

print("Recompensa {}".format(episode_reward))

Recompensa 64.0
